In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import LOCM2
from utils import read_plan
from evaluator import ExecutabilityEvaluator

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

In [2]:
def test_exe(domain_name, trace, test_trace=None):
    try:
        locm2 = LOCM2(state_param=True, viz=True)
        model = locm2.extract_model([trace])

        actions = model.actions
        
        pddl_domain = model.to_pddl_domain("test_bw")
        pddl_domain.dump()
        evaluator = ExecutabilityEvaluator(pddl_domain, 'overall', debug=True)
        if test_trace is not None:
            exe = evaluator.get_overall_executability('l', test_trace, set(), set())
        else:
            exe = evaluator.get_overall_executability('l', trace, set(), set())
        if exe < 1:
            print(f"Warning! domain: {domain_name} - executability: {exe}")
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [ ]:
plan = "(make_sandwich sandw1?sandwich bread1?bread-portion content1?content-portion),(make_sandwich sandw2?sandwich bread2?bread-portion content2?content-portion),(make_sandwich sandw3?sandwich bread3?bread-portion content3?content-portion),(make_sandwich sandw4?sandwich bread4?bread-portion content4?content-portion),(put_on_tray sandw1?sandwich tray1?tray),(put_on_tray sandw2?sandwich tray1?tray),(put_on_tray sandw3?sandwich tray1?tray),(put_on_tray sandw4?sandwich tray1?tray),(move_tray tray1?tray kitchen?place table1?place),(serve_sandwich sandw1?sandwich child1?child tray1?tray table1?place),(serve_sandwich sandw2?sandwich child4?child tray1?tray table1?place),(move_tray tray1?tray table1?place table2?place),(serve_sandwich sandw3?sandwich child2?child tray1?tray table2?place),(move_tray tray1?tray table2?place table3?place),(serve_sandwich sandw4?sandwich child3?child tray1?tray table3?place)"
test_plan= "(unstack b3?object b2?object),(putdown b3?object),(unstack b2?object b1?object),(putdown b2?object),(pickup b2?object),(stack b2?object b3?object),(pickup b1?object),(stack b1?object b2?object)"

trace = read_plan(plan)
test_trace = read_plan(test_plan)

test_exe("test_bw", trace)
# plan2 = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
# test_exe("test_bw", read_plan(plan2))

Problem test_bw: None [:strips, :typing]
Types:
  object
  bread3: s1
  bread4: s1
  bread2: s1
  bread1: s1
  kitchen: s2
  table2: s2
  table3: s2
  table1: s2
  child1: s3
  child4: s3
  child2: s3
  child3: s3
  sandw3: s4
  sandw2: s4
  sandw4: s4
  sandw1: s4
  tray1: s5
  content4: s6
  content2: s6
  content3: s6
  content1: s6
  zero: s0
Predicates:
  s3f0_state0(?x0: s3)
  s2f0_state2(?x0: s2)
  s4f0_state2(?x0: s4)
  s2f0_state0(?x0: s2)
  s1f0_state0(?x0: s1)
  s6f0_state0(?x0: s6)
  s6f0_state1(?x0: s6)
  s4f0_state0(?x0: s4, ?x1: s5)
  s0f0_state0(?x0: s0)
  s4f0_state3(?x0: s4)
  s5f1_state0(?x0: s5)
  s5f0_state1(?x0: s5)
  s1f0_state1(?x0: s1)
  s2f0_state1(?x0: s2)
  s5f0_state0(?x0: s5)
  s4f0_state1(?x0: s4)
  s3f0_state1(?x0: s3)
Functions:
Actions:
put_on_tray(?x0: s0, ?x1: s4, ?x2: s5)
Precondition:
  Conjunction
    Atom s0f0_state0(?x0)
    Atom s5f1_state0(?x2)
    Atom s4f0_state2(?x1)
    Atom s5f0_state0(?x2)
Effects:
  Atom s4f0_state0(?x1, ?x2)
  NegatedA

1.0

In [4]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         details = line.split("&&")
#         name = f"{details[0]}-{details[2]}-{details[3]}"
      
#         plan = details[-1]
#         trace = read_plan(plan)
#         test_exe(name, trace)